In [11]:
import asyncio
import websockets
import json
import requests
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
load_dotenv()

{'_index': 'coinbase-info', '_type': '_doc', '_id': 'FBXeFYwBHu8B-D6xGNcY', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22541, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'iibeFYwBiNwh5hwHGAK6', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22542, '_primary_term': 1}


In [12]:
api="""/_settings"""
data= """{
    "settings": {
        "index.mapping.total_fields.limit": 40
    }
}"""

url = 'https://0516a1b9b1d146d68246553b2f420a3f.northeurope.azure.elastic-cloud.com:443/'
username = os.getenv("ELASTIC_USERNAME")
password = os.getenv("ELASTIC_PASSWORD")
index='coinbase-info'
headers = {"Content-Type": "application/json"}

response = requests.put(url+index+api, data=data, headers=headers, auth=(username, password))
res = response.json()
print(res)

{'acknowledged': True}


{'_index': 'coinbase-info', '_type': '_doc', '_id': 'FRXeFYwBHu8B-D6xGddt', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22543, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'FhXeFYwBHu8B-D6xGtcR', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22544, '_primary_term': 1}


{'_index': 'coinbase-info', '_type': '_doc', '_id': 'GBXeFYwBHu8B-D6xHdfx', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22550, '_primary_term': 1}
{'acknowledged': True}


In [13]:
api="""/_mapping"""
data= """{
    "properties": {
        "time": {
            "type": "date"
        },
        "price": {
            "type": "double"
        },
        "open_24h": {
            "type": "double"
        },
        "volume_24h": {
            "type": "double"
        },
        "high_24h": {
            "type": "double"
        },
        "low_24h_price": {
            "type": "double"
        },
        "volume_30d": {
            "type": "double"
        },
        "best_bid": {
            "type": "double"
        },
        "best_bid_size": {
            "type": "double"
        },
        "best_ask": {
            "type": "double"
        },
        "best_ask_size": {
            "type": "double"
        },
        "side_w": {
            "type": "keyword"
        },
        "last_size": {
            "type": "double"
        }
    }
}"""
response = requests.put(url+index+api, data=data, headers=headers, auth=(username, password))
res = response.json()
res

{'acknowledged': True}

In [14]:
def convert_timestamp(f):
    datetime_object = datetime.strptime(f, '%Y-%m-%dT%H:%M:%S.%fZ')
    return datetime_object.strftime("%Y-%m-%dT%H:%M:%SZ")

In [ ]:
api="""/_doc"""
url = 'https://0516a1b9b1d146d68246553b2f420a3f.northeurope.azure.elastic-cloud.com:443/'
username = os.getenv("ELASTIC_USERNAME")
password = os.getenv("ELASTIC_PASSWORD")
index='coinbase-info'
headers = {"Content-Type": "application/json"}

async def coinbase_websocket():
    uri = "wss://ws-feed.exchange.coinbase.com"

    async with websockets.connect(uri) as websocket:
        subscription_message = {
            "type": "subscribe",
            "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
        }

        await websocket.send(json.dumps(subscription_message))

        end_time = datetime.utcnow() + timedelta(hours=5)
        first_message_received = False

        while datetime.utcnow() < end_time:
            response = await websocket.recv()

            if not first_message_received:
                first_message_received = True
                continue

            try:
                ticker_data = json.loads(response)
                time_value = ticker_data.get("time")

                converted_time = convert_timestamp(time_value)

                for key, value in ticker_data.items():
                    if isinstance(value, str):
                        ticker_data[key] = value.replace("‘", '"')

                filtered_data = {
                    "price": ticker_data.get("price"),
                    "open_24h": ticker_data.get("open_24h"),
                    "volume_24h": ticker_data.get("volume_24h"),
                    "low_24h_price": ticker_data.get("low_24h"),
                    "high_24h": ticker_data.get("high_24h"),
                    "volume_30d": ticker_data.get("volume_30d"),
                    "best_bid": ticker_data.get("best_bid"),
                    "best_bid_size": ticker_data.get("best_bid_size"),
                    "best_ask": ticker_data.get("best_ask"),
                    "best_ask_size": ticker_data.get("best_ask_size"),
                    "side_w": ticker_data.get("side"),
                    "time": converted_time,
                    "last_size": ticker_data.get("last_size"),
                }

                filtered_data_json = json.dumps(filtered_data)
                data_string = filtered_data_json.replace(", ", ",\n    ")
                #print("Modified, filtered, and quotes replaced JSON data:", data_string)
                response = requests.post(url+index+api, data=data_string, headers=headers, auth=(username, password))
                print(response.json())
                

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON response: {e}")

if asyncio.get_event_loop().is_running():
    asyncio.ensure_future(coinbase_websocket())
else:
    asyncio.run(coinbase_websocket())


{'_index': 'coinbase-info', '_type': '_doc', '_id': 'GRXeFYwBHu8B-D6xIddL', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22551, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'jybeFYwBiNwh5hwHIQLn', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22552, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'kCbeFYwBiNwh5hwHIwJ_', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22553, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'kSbeFYwBiNwh5hwHJQJQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22554, '_primary_term': 1}
{'_index': 'coinbase-info', '_type': '_doc', '_id': 'GhXeFYwBHu8B-D6xJdf5', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22

Task exception was never retrieved
future: <Task finished name='Task-12' coro=<coinbase_websocket() done, defined at /var/folders/mw/2cx6x93d04g09y_zdvpb9vpc0000gn/T/ipykernel_3859/159664652.py:8> exception=ConnectionClosedError(None, Close(code=<CloseCode.INTERNAL_ERROR: 1011>, reason='keepalive ping timeout'), None)>
Traceback (most recent call last):
  File "/Users/santi/anaconda3/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1301, in close_connection
  File "/Users/santi/anaconda3/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 974, in transfer_data
    # Wait until there's room in the queue (if necessary).
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/mw/2cx6x93d04g09y_zdvpb9vpc0000gn/T/ipykernel_3859/159664652.py", line 23, in coinbase_websocket
    response = await webs

{'_index': 'coinbase-info', '_type': '_doc', '_id': '9ibtFYwBiNwh5hwH5Qw0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 27152, '_primary_term': 1}
